MIAS-breast

In [ ]:
import os
import cv2
import numpy as np

def np_CountUpContinuingOnes(b_arr):
    left = np.arange(len(b_arr))
    left[b_arr > 0] = 0
    left = np.maximum.accumulate(left)
    rev_arr = b_arr[::-1]
    right = np.arange(len(rev_arr))
    right[rev_arr > 0] = 0
    right = np.maximum.accumulate(right)
    right = len(rev_arr) - 1 - right[::-1]
    return right - left - 1

def ExtractBreast(img):
    img_copy = img.copy()
    img = np.where(img <= 20, 0, img)
    height, _ = img.shape
    y_a = height // 2 + int(height * 0.4)
    y_b = height // 2 - int(height * 0.4)
    b_arr = img[y_b:y_a].std(axis=0) != 0
    continuing_ones = np_CountUpContinuingOnes(b_arr)
    col_ind = np.where(continuing_ones == continuing_ones.max())[0]
    img = img[:, col_ind]
    _, width = img.shape
    x_a = width // 2 + int(width * 0.4)
    x_b = width // 2 - int(width * 0.4)
    b_arr = img[:, x_b:x_a].std(axis=1) != 0
    continuing_ones = np_CountUpContinuingOnes(b_arr)
    row_ind = np.where(continuing_ones == continuing_ones.max())[0]
    return img_copy[row_ind][:, col_ind]

# 定义路径
TXT_PATH = '/Volumes/图图/MIAS/archive/Info.txt'
PGM_PATH = '/Volumes/图图/MIAS/archive/all-mias'
OUTPUT_BASE_PATH = 'Benchmark/MIAS-breast'

# 读取TXT文件
with open(TXT_PATH, 'r') as file:
    lines = file.readlines()

# 跳过标题行
lines = lines[1:]

# 初始化一个字典，用于合并相同的 refnum 信息
merged_data = {}

for line in lines:
    parts = line.split()
    refnum = parts[0]
    bg = str(parts[1]).replace(' ', '')
    cls = str(parts[2]).replace(' ', '')
    severity = str(parts[3]).replace(' ', '') if len(parts) > 3 else 'N'
    if refnum in merged_data:
        merged_data[refnum]['class'].add(cls)
        if severity == 'M':
            merged_data[refnum]['severity'] = 'M'
    else:
        merged_data[refnum] = {
            'background': bg,
            'class': {cls},
            'severity': severity
        }

def process_and_save_all(data):
    for refnum, info in data.items():
        severity = info['severity']
        cancer = 'Yes' if severity == 'M' else 'No'

        # 读取 PGM 文件
        pgm_file = os.path.join(PGM_PATH, refnum + '.pgm')
        if os.path.exists(pgm_file):
            img = cv2.imread(pgm_file, cv2.IMREAD_GRAYSCALE)
            img = ExtractBreast(img)
            img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

            # 创建输出目录
            output_dir = os.path.join(OUTPUT_BASE_PATH, refnum)
            os.makedirs(output_dir, exist_ok=True)

            # 保存图像为 JPEG 格式
            jpg_output_path = os.path.join(output_dir, 'img.jpg')
            cv2.imwrite(jpg_output_path, img)

            print(f"Processed {refnum}")
        else:
            print(f"PGM file for {refnum} not found.")

# 处理并保存所有数据
process_and_save_all(merged_data)

MIAS-finding

In [ ]:
import os
import cv2
import numpy as np

def crop_and_save(img, x, y, radius, output_path):
    h, w = img.shape
    x = int(x)
    y = w - int(y)  # Convert y-coordinate from DICOM to image coordinates
    radius = int(radius)
    x1 = max(x - radius, 0)
    y1 = max(y - radius, 0)
    x2 = min(x + radius, img.shape[1])
    y2 = min(y + radius, img.shape[0])

    cropped_img = img[y1:y2, x1:x2]
    cropped_img = cv2.normalize(cropped_img, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    cv2.imwrite(output_path, cropped_img)

# Define paths
TXT_PATH = '/Volumes/图图/MIAS/archive/Info.txt'
PGM_PATH = '/Volumes/图图/MIAS/archive/all-mias'
OUTPUT_BASE_PATH = 'Benchmark/MIAS-finding'

# Read TXT file
with open(TXT_PATH, 'r') as file:
    lines = file.readlines()

# Skip header line and filter lines
lines = lines[1:]
lines = [line for line in lines if len(line.split()) > 4]

def process_and_save_all(lines):
    refnum_counter = {}
    for line in lines:
        parts = line.split()
        refnum = parts[0]
        cls = str(parts[2]).replace(' ', '')
        severity = str(parts[3]).replace(' ', '')
        x, y, radius = parts[4], parts[5], parts[6]

        # Handle duplicate refnums
        if refnum in refnum_counter:
            refnum_counter[refnum] += 1
        else:
            refnum_counter[refnum] = 1
        refnum_with_suffix = f"{refnum}_{refnum_counter[refnum]}"

        # Read PGM file
        pgm_file = os.path.join(PGM_PATH, refnum + '.pgm')
        if os.path.exists(pgm_file):
            img = cv2.imread(pgm_file, cv2.IMREAD_GRAYSCALE)
            output_dir = os.path.join(OUTPUT_BASE_PATH, refnum_with_suffix)
            os.makedirs(output_dir, exist_ok=True)

            # Save cropped image
            jpg_output_path = os.path.join(output_dir, 'img.jpg')
            crop_and_save(img, x, y, radius, jpg_output_path)


            print(f"Processed {refnum_with_suffix}")
        else:
            print(f"PGM file for {refnum} not found.")

# Process and save all data
process_and_save_all(lines)

generate normal case

In [ ]:
import os
import cv2
import numpy as np
import random

def find_cropped_image_position(full_image, cropped_image):
    res = cv2.matchTemplate(full_image, cropped_image, cv2.TM_CCOEFF_NORMED)
    _, _, _, max_loc = cv2.minMaxLoc(res)
    top_left = max_loc
    bottom_right = (top_left[0] + cropped_image.shape[1], top_left[1] + cropped_image.shape[0])
    return (*top_left, *bottom_right)

def random_crop_outside_bboxes(full_image, bboxes, crop_size, num_crops=1):
    height, width = full_image.shape[:2]
    cropped_images = []
    
    def is_inside_any_bbox(x, y):
        for (x1, y1, x2, y2) in bboxes:
            if x1 <= x < x2 and y1 <= y < y2:
                return True
        return False
    
    for _ in range(num_crops):
        while True:
            x = random.randint(0, width - crop_size[1])
            y = random.randint(0, height - crop_size[0])
            if not is_inside_any_bbox(x, y):
                cropped_img = full_image[y:y + crop_size[0], x:x + crop_size[1]]
                cropped_images.append(cropped_img)
                break
    return cropped_images

def process_all_crops_for_folder(full_image_path, cropped_folders, output_dir):
    full_image = cv2.imread(full_image_path+'/img.jpg')
    bboxes = []

    # Collect bboxes from all crops
    try:
        for folder in cropped_folders:
            for cropped_image_name in os.listdir(folder):
                if cropped_image_name.endswith('img.jpg'):
                    cropped_image_path = os.path.join(folder, cropped_image_name)
                    cropped_image = cv2.imread(cropped_image_path)
                    bbox = find_cropped_image_position(full_image, cropped_image)
                    bboxes.append(bbox)

        # Generate random crops and update info_dict.npy
        for idx,folder in enumerate(cropped_folders):

            crop_size = (bboxes[idx][3] - bboxes[idx][1], bboxes[idx][2] - bboxes[idx][0])
            random_crops = random_crop_outside_bboxes(full_image, bboxes, crop_size)
            for crop in random_crops:
                normal_folder = os.path.join(output_dir, f"{os.path.basename(folder)}_normal")
                os.makedirs(normal_folder, exist_ok=True)
                normal_image_path = os.path.join(normal_folder, 'img.jpg')
                cv2.imwrite(normal_image_path, crop)
    except Exception as e:
        print(f"An error occurred: {e}")
        print(full_image_path+' OOps!')
        

root_dir = 'Benchmark/MIAS-finding'
base_image_dir = 'Benchmark/MIAS-breast'
output_dir = 'Benchmark/MIAS-finding'

def find_full_image_path(base_dir, full_image_key):
    full_image_path = os.path.join(base_dir, full_image_key)
    if os.path.exists(full_image_path):
        return full_image_path
    else:
        return None

# Map cropped images to their respective full images
full_image_to_crops_map = {}
for subdir in os.listdir(root_dir):
    full_image_key = '_'.join(subdir.split('_')[:-1])
    full_image_to_crops_map.setdefault(full_image_key, []).append(os.path.join(root_dir, subdir))

# Process all mapped cropped folders for each full image
for full_image_key, folders in full_image_to_crops_map.items():
    # patient_id, side, view = full_image_key.split('_')
    full_image_path = find_full_image_path(base_image_dir, full_image_key)
    if full_image_path:
        process_all_crops_for_folder(full_image_path, folders, output_dir)
    else:
        print(f"Full image not found for {full_image_key}")
